Importação das bibliotecas que serão usadas

In [ ]:
import pandas as pd

In [ ]:
import joblib

In [ ]:
from scipy.stats import uniform, loguniform,randint

In [ ]:
from statsmodels.stats.multicomp import MultiComparison

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import sklearn

In [ ]:
import plotly.express as px

Abrindo a base de dados de treino e teste

In [ ]:
base_treino=pd.read_csv('data/train.csv')

In [ ]:
base_treino

In [ ]:
base_teste=pd.read_csv('data/test.csv')

In [ ]:
base_teste

Visualização dos dados e verificações utilizando isnull nos data frames para caçar dados inconsistentes,nulos e outliers

In [ ]:
base_teste.describe()

In [ ]:
base_treino.describe()

In [ ]:
base_teste.isnull().sum()

In [ ]:
base_treino.isnull().sum()

Vemos que não há dados nulos nessas duas tabelas

Como a base de dados é para previsão de regressão ao invés de classificação então não devemos preocupar com desbalanceamento de dados classificadores , se no caso deveriamos fazer undersampling ou oversampling

Partiremos para visualização dos dados

In [ ]:
base_treino.info()

In [ ]:
base_teste.info()

In [ ]:
print(base_treino['POSTED_BY'].unique())
print(base_treino['BHK_OR_RK'].unique())
print(len(base_treino['ADDRESS'].unique()))
print(len(base_treino[base_treino['POSTED_BY']=='Owner']))
print(len(base_treino[base_treino['POSTED_BY']=='Dealer']))

A coluna 'BHK_OR_RK' pode ser dropada pois há muitos valores a mais de BHK do que RK

In [ ]:
base_treino=base_treino.drop(columns=['BHK_OR_RK'],axis=1)
base_teste=base_teste.drop(columns=['BHK_OR_RK'],axis=1)

Acabei por remover ADDRESS e POSTED_BY pois gerou melhores resultados no treinamento 

In [ ]:
base_treino=base_treino.drop(columns=['ADDRESS'],axis=1)
base_teste=base_teste.drop(columns=['ADDRESS'],axis=1)
base_treino=base_treino.drop(columns=['POSTED_BY'],axis=1)
base_teste=base_teste.drop(columns=['POSTED_BY'],axis=1)

É visto que algumas variáveis como 'UNDER_CONSTRUCTION' E 'RERA' por possuírem valores 0 ou 1 não possuem outliers

In [ ]:
fig=plt.Figure(figsize=(10,15))
sns.scatterplot(x=base_treino['UNDER_CONSTRUCTION'],y=base_treino['RERA'])
plt.show()

E a variável 'BHK_NO' possue valores de 1 até 20 , possuindo um certo outlier visto que a concentração de dados está nos valores de 1 até 4, sendo na base de teste e treino valores bem insignificantes maiores que esse intervalo de 1 a 4 , não vale a pena retirá los por serem valores bem baixo (menos de 1% da base)

In [ ]:
base_teste[base_teste['BHK_NO.']>5]

In [ ]:
base_treino[base_treino['BHK_NO.']>5]

In [ ]:
fig=plt.Figure(figsize=(10,15))
sns.countplot(x=base_treino['BHK_NO.'])
plt.show()

In [ ]:
grafico1=px.box(y=base_treino['SQUARE_FT'])
grafico1.show()

In [ ]:
media_squareft=base_treino['SQUARE_FT'].mean()
media_squareft

In [ ]:
tamanho_antes_de_remover_outliers_square_ft=len(base_treino)

In [ ]:
base_treino=base_treino[base_treino['SQUARE_FT']<=2527.114]

In [ ]:
tamanho_depois_de_remover_outliers_square_ft=len(base_treino)

In [ ]:
((tamanho_antes_de_remover_outliers_square_ft-tamanho_depois_de_remover_outliers_square_ft)/tamanho_antes_de_remover_outliers_square_ft)*100

Esses valores são muito acima do upper fance , é melhor remover eles da base de dados para não ocasionar em pesos serem calculados de forma errônea

In [ ]:
media_squareft=base_treino['SQUARE_FT'].mean()
media_squareft

A média saiu de 19802 para 1191 quando retirado os valores considerados outliers , e assim ainda mantendo 94.45% do tamanho da base antes de remover , assim não sendo um valor muito alto de remoção e que vai ajudar bastante as predições

In [ ]:
grafico1=px.box(y=base_treino['SQUARE_FT'])
grafico1.show()

Olhemos agora latitude e longitude

In [ ]:
fig=plt.Figure(figsize=(10,15))
sns.scatterplot(x=base_treino['LATITUDE'],y=base_treino['LONGITUDE'])
plt.show()

In [ ]:
base_treino[base_treino['LATITUDE']>60]

In [ ]:
len(base_treino)

A coluna de latitude estã mais concentrada nos valores entre 60 e 100 , onde valores menores que esses são considerados outliers e sendo apenas 143 registros , para manter a estabilidade do modelo podemos retirar esses valores

In [ ]:
base_treino=base_treino[base_treino['LATITUDE']>60]

In [ ]:
fig=plt.Figure(figsize=(10,15))
sns.scatterplot(x=base_treino['LATITUDE'],y=base_treino['LONGITUDE'])
plt.show()

In [ ]:
base_treino=base_treino[base_treino['LATITUDE']<100]

In [ ]:
fig=plt.Figure(figsize=(10,15))
sns.scatterplot(x=base_treino['LATITUDE'],y=base_treino['LONGITUDE'])
plt.show()

In [ ]:
grafico1=px.box(y=base_treino['TARGET(PRICE_IN_LACS)'])
grafico1.show()

Vemos que agora a base está bem balanceada e corrigida em relação a outliers e dados incosistentes, logo podemos ir para a transformação dos dados

In [ ]:
base_treino.info()

Agora será feito o treinamento com a base de dados retirando ruídos

In [ ]:
base_treino

In [ ]:
base_teste

In [ ]:
X_treino=base_treino.iloc[:,[0,1,2,3,4,5,6,7]].values
X_teste=base_teste.iloc[:,[0,1,2,3,4,5,6,7]].values
Y=base_treino['TARGET(PRICE_IN_LACS)'].values

In [ ]:
X_treino

In [ ]:
column_transformer=sklearn.compose.ColumnTransformer(
    transformers=[
        ('Scaler',sklearn.preprocessing.StandardScaler(),[0,1,2,3,4,5,6,7])]
,remainder='passthrough')

In [ ]:
X_treino=column_transformer.fit_transform(X_treino)

In [ ]:
X_teste=column_transformer.transform(X_teste)

In [ ]:
X_treino.shape,X_teste.shape

In [ ]:
Y.shape

In [ ]:
scaler=sklearn.preprocessing.StandardScaler()

In [ ]:
Y=scaler.fit_transform(Y.reshape(-1,1))

In [ ]:
X_treino

In [ ]:
X_teste

Agora separamos também os dados de forma completa sem retirar os ruidos mas retirando 'POSTED_BY','ADRESS' e 'BHK_OR_RK'

In [ ]:
X_treino_com_ruido=pd.read_csv('data/train.csv')

In [ ]:
X_treino_com_ruido.drop(columns=['BHK_OR_RK','ADDRESS','POSTED_BY'],axis=1,inplace=True)

In [ ]:
X_treino_com_ruido

In [ ]:
X_treino_com_ruido=X_treino_com_ruido.iloc[:,:8].values

In [ ]:
X_treino_com_ruido=column_transformer.fit_transform(X_treino_com_ruido)

In [ ]:
Y_com_ruido=pd.read_csv('data/train.csv')

In [ ]:
Y_com_ruido=Y_com_ruido['TARGET(PRICE_IN_LACS)'].values

In [ ]:
Y_com_ruido=scaler.fit_transform(Y_com_ruido.reshape(-1,1))

Agora que a base de dados de treino ,teste e o target estão separados podemos partir para o grid search onde veremos os melhores parametros para cada modelo e depois compararemos os desempenhos dos modelos

In [ ]:
parametros_decision_tree={'max_depth':[5]}

In [ ]:
grid_search_decision_three=sklearn.model_selection.GridSearchCV(estimator=sklearn.tree.DecisionTreeRegressor(),param_grid=parametros_decision_tree,verbose=2)

In [ ]:
grid_search_decision_three.fit(X_treino,Y)

In [ ]:
grid_search_decision_three.best_params_

In [ ]:
grid_search_decision_three.best_score_

In [ ]:
parametros_knn={'n_neighbors':[3,5,10,20,25],'weights':['uniform','distance'],'p':[1,2],'algorithm':['auto','ball_tree', 'kd_tree']}

In [ ]:
grid_search_knn=sklearn.model_selection.GridSearchCV(estimator=sklearn.neighbors.KNeighborsRegressor(),param_grid=parametros_knn,verbose=2)

In [ ]:
grid_search_knn.fit(X_treino,Y)

In [ ]:
grid_search_knn.best_params_

In [ ]:
best_params_knn=grid_search_knn.best_params_

In [ ]:
grid_search_knn.best_score_

In [ ]:
parametros_random_forest={'criterion':['squared_error'],'n_estimators':[40,60],'min_samples_split':[2,5,7],'min_samples_leaf':[1,5,7]}

In [ ]:
grid_search_random_forest=sklearn.model_selection.GridSearchCV(estimator=sklearn.ensemble.RandomForestRegressor(),param_grid=parametros_random_forest,verbose=2)

In [ ]:
grid_search_random_forest.fit(X_treino,Y.ravel())

In [ ]:
grid_search_random_forest.best_params_

In [ ]:
grid_search_random_forest.best_score_

In [ ]:
parametros_regressao_linear={
    'fit_intercept': [True, False]
}

In [ ]:
grid_search_regressao_linear=sklearn.model_selection.GridSearchCV(estimator=sklearn.linear_model.LinearRegression(),param_grid=parametros_regressao_linear,verbose=2)

In [ ]:
poly=sklearn.preprocessing.PolynomialFeatures(degree=3)

In [ ]:
grid_search_regressao_linear.fit(poly.fit_transform(X_treino),Y.ravel())

In [ ]:
grid_search_regressao_linear.best_params_

In [ ]:
grid_search_regressao_linear.best_score_

In [ ]:
param_dist_svm = {
    'C': loguniform(1e-2, 1e2),      
    'epsilon': uniform(0.01, 1.0),   
    'kernel': ['rbf', 'poly'],
    'gamma': ['auto']       
}

In [ ]:
randozimed_search_svm=sklearn.model_selection.RandomizedSearchCV(estimator=sklearn.svm.SVR(),n_iter=25,scoring='r2',cv=3,verbose=2,n_jobs=-1,random_state=42,param_distributions=param_dist_svm)

In [ ]:
randozimed_search_svm.fit(X_treino,Y.ravel())

In [ ]:
randozimed_search_svm.best_params_

In [ ]:
randozimed_search_svm.best_score_

In [ ]:
param_dist_rede_neural = {
    'hidden_layer_sizes': [(randint.rvs(50, 200),),      # 1 camada oculta
                           (randint.rvs(50, 200), randint.rvs(20, 100)),  # 2 camadas
                           (randint.rvs(50, 200), randint.rvs(20, 100), randint.rvs(10, 50))], # 3 camadas
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': uniform(1e-5, 1e-2),   # regularização L2
    'learning_rate_init': uniform(1e-4, 1e-2),
}

In [ ]:
random_search_rede_neural = sklearn.model_selection.RandomizedSearchCV(
    estimator=sklearn.neural_network.MLPRegressor(max_iter=1000,random_state=42),
    param_distributions=param_dist_rede_neural,
    n_iter=20,         
    scoring='r2',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

In [ ]:
random_search_rede_neural.fit(X_treino,Y.ravel())

In [ ]:
random_search_rede_neural.best_params_

In [ ]:
random_search_rede_neural.best_score_

Depois de fazer o GridSearch e RandomizedSearch podemos partir para as validações cruzadas utilizando o k-folds

In [ ]:
resultados_decision_tree=[]
resultados_knn=[]
resultados_linear_regression=[]
resultados_SVR=[]
resultados_rede_neural=[]
resultados_random_forest=[]
for i in range(15):
    k_fold=sklearn.model_selection.KFold(n_splits=2,shuffle=True,random_state=i)
    knn=sklearn.neighbors.KNeighborsRegressor(algorithm='ball_tree', n_neighbors=20, p= 1, weights='distance')
    decision_tree=sklearn.tree.DecisionTreeRegressor(max_depth=5)
    linear_regression=sklearn.linear_model.LinearRegression(fit_intercept =False)
    svr=sklearn.svm.SVR(C=np.float64(85.68869785189007),epsilon=np.float64(0.47676289324797994),gamma='auto',kernel='rbf')
    rede_neural=sklearn.neural_network.MLPRegressor(activation='tanh',alpha=np.float64(0.0009867211400638386),hidden_layer_sizes=(142, 88, 39),learning_rate_init=np.float64(0.006932635188254582),solver='sgd')
    random_forest=sklearn.ensemble.RandomForestRegressor(criterion='squared_error',min_samples_leaf=1,min_samples_split=5,n_estimators=60)
    scores_decision_tree=sklearn.model_selection.cross_val_score(decision_tree,X_treino,Y.ravel(),cv=k_fold)
    scores_knn=sklearn.model_selection.cross_val_score(knn,X_treino,Y.ravel(),cv=k_fold)
    scores_linear_regression=sklearn.model_selection.cross_val_score(linear_regression,X_treino,Y.ravel(),cv=k_fold)
    scores_svr=sklearn.model_selection.cross_val_score(svr,X_treino,Y.ravel(),cv=k_fold)
    scores_rede_neural=sklearn.model_selection.cross_val_score(rede_neural,X_treino,Y.ravel(),cv=k_fold)
    scores_random_forest=sklearn.model_selection.cross_val_score(random_forest,X_treino,Y.ravel(),cv=k_fold)
    resultados_decision_tree.append(scores_decision_tree.mean())
    resultados_knn.append(scores_knn.mean())
    resultados_linear_regression.append(scores_linear_regression.mean())
    resultados_SVR.append(scores_svr.mean())
    resultados_rede_neural.append(scores_rede_neural.mean())
    resultados_random_forest.append(scores_random_forest.mean())
    print(i)
    

Colocamos os resultados em um data frame para uma boa visualização e depois em uma lista para ser utilizado no MultiComparison

In [ ]:
data_frame=pd.DataFrame({'SVR':resultados_SVR,'KNN':resultados_knn,'Regressao linear':resultados_linear_regression,'Random Forest':resultados_random_forest,'Rede neural':resultados_rede_neural,'Árvore de decisao':resultados_decision_tree})

In [ ]:
data_frame

In [ ]:
lista=['SVR','KNN','Regressao linear','Random Forest','Rede neural','Árvore de decisao']
algoritmos=[]
resultados=[]

In [ ]:
cont=0
indice=0
for i in range(0,90):
    if i%15==0 and i!=0:
        cont+=1
    algoritmos.append(lista[cont])
    if(indice==15):
        indice=0
    resultados.append(data_frame.iloc[indice,cont])
    indice+=1
    


In [ ]:
resultados_concatenados={'accuracy':resultados,'algoritmos':algoritmos}

In [ ]:
resultados_concatenados

In [ ]:
data_frame_algoritmos=pd.DataFrame(resultados_concatenados)

In [ ]:
data_frame_algoritmos

In [ ]:
compara_algoritmos=MultiComparison(data_frame_algoritmos['accuracy'],data_frame_algoritmos['algoritmos'])

In [ ]:
compara_algoritmos.tukeyhsd().plot_simultaneous()

Esse foi o teste com a base de dados retirando outliers , agora utilizarei com a base cheia

In [ ]:
resultados_decision_tree=[]
resultados_knn=[]
resultados_linear_regression=[]
resultados_SVR=[]
resultados_rede_neural=[]
resultados_random_forest=[]
resultados_gbr=[]
for i in range(15):
    k_fold=sklearn.model_selection.KFold(n_splits=2,shuffle=True,random_state=i)
    gbr=sklearn.ensemble.GradientBoostingRegressor(random_state=69)
    knn=sklearn.neighbors.KNeighborsRegressor(algorithm='ball_tree', n_neighbors=20, p= 1, weights='distance')
    decision_tree=sklearn.tree.DecisionTreeRegressor(max_depth=5)
    linear_regression=sklearn.linear_model.LinearRegression(fit_intercept =False)
    svr=sklearn.svm.SVR(C=np.float64(85.68869785189007),epsilon=np.float64(0.47676289324797994),gamma='auto',kernel='rbf')
    rede_neural=sklearn.neural_network.MLPRegressor(activation='tanh',alpha=np.float64(0.0009867211400638386),hidden_layer_sizes=(142, 88, 39),learning_rate_init=np.float64(0.006932635188254582),solver='sgd')
    random_forest=sklearn.ensemble.RandomForestRegressor(criterion='squared_error',min_samples_leaf=1,min_samples_split=5,n_estimators=60)
    scores_decision_tree=sklearn.model_selection.cross_val_score(decision_tree,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_gbr=sklearn.model_selection.cross_val_score(gbr,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_knn=sklearn.model_selection.cross_val_score(knn,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_linear_regression=sklearn.model_selection.cross_val_score(linear_regression,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_svr=sklearn.model_selection.cross_val_score(svr,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_rede_neural=sklearn.model_selection.cross_val_score(rede_neural,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    scores_random_forest=sklearn.model_selection.cross_val_score(random_forest,X_treino_com_ruido,Y_com_ruido.ravel(),cv=k_fold)
    resultados_decision_tree.append(scores_decision_tree.mean())
    resultados_knn.append(scores_knn.mean())
    resultados_linear_regression.append(scores_linear_regression.mean())
    resultados_SVR.append(scores_svr.mean())
    resultados_rede_neural.append(scores_rede_neural.mean())
    resultados_random_forest.append(scores_random_forest.mean())
    resultados_gbr.append(scores_gbr.mean())
    print(i)

In [ ]:
data_frame=pd.DataFrame({'SVR':resultados_SVR,'KNN':resultados_knn,'Regressao linear':resultados_linear_regression,'Random Forest':resultados_random_forest,'Rede neural':resultados_rede_neural,'Árvore de decisao':resultados_decision_tree,'GBR':resultados_gbr})

In [ ]:
lista=['SVR','KNN','Regressao linear','Random Forest','Rede neural','Árvore de decisao','GBR']
algoritmos=[]
resultados=[]

In [ ]:
cont=0
indice=0
for i in range(0,105):
    if i%15==0 and i!=0:
        cont+=1
    algoritmos.append(lista[cont])
    if(indice==15):
        indice=0
    resultados.append(data_frame.iloc[indice,cont])
    indice+=1

In [ ]:
resultados_concatenados={'accuracy':resultados,'algoritmos':algoritmos}

In [ ]:
data_frame_algoritmos=pd.DataFrame(resultados_concatenados)

In [ ]:
data_frame_algoritmos=data_frame_algoritmos[data_frame_algoritmos['accuracy']>0]

In [ ]:
data_frame_algoritmos

In [ ]:
compara_algoritmos=MultiComparison(data_frame_algoritmos['accuracy'],data_frame_algoritmos['algoritmos'])

In [ ]:
compara_algoritmos.tukeyhsd().plot_simultaneous()

No final podemos perceber que o modelo GradientBoostingRegressor obteve resultados melhores do que os demais,além de que o treinamento utilizando a base com outliers obteve uma precisão muito superior a base sem outliers,isso mostrando que os outliers tinha uma parte importante para modelar melhor os pesos e gerar um resultado além do esperado.

O GradientBoostingRegressor não passou por Grid ou RandomizedSearch pois foi só para uso de testes e acabou sendo superior aos demais modelos, até porque o teste dos demais foi de grande demora computacional e mais um modelo que possui muita parametrização iria custar bantante tempo para esse projeto que considerei pequeno por ser o primeiro feito por mim após meus estudos sobre ML e dados.